In [1]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive/')

# 目的の場所（フォルダ・ディレクトリ）へ移動（各自の環境で適宜修正）
%cd "/content/drive/MyDrive/Colab Notebooks/JKJ1A/"
%ls

Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/JKJ1A
backyard/  data/  model/  notebook/  notebook_ans/  src/  src_ans/


---
# 準備

In [2]:
# モジュールのインポート
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt 

import warnings
warnings.filterwarnings("ignore")  # warningを表示しない

In [3]:
import sys
sys.path.append('src')  # src/にあるコードを利用できるようにする

from models import VGG
from cifar10 import load_data

### 復習問題
- Day1のEX課題1で，CIFAR-10で学習したVGGモデルを学習した．このモデルを`model`という名前で読み込め（Day1-1のモデルの保存のところで，読み込み方法も載せてある）．
- `load_data()`関数を利用してCIFAR10を読みこめ．これで`trainloader`, `testloader`, `classes`が準備される．`batch_size`は4とせよ．
- testloaderからミニバッチ`x`と`y`をひと組取り出せ（GPUにも移すこと）

#### 解答

In [ ]:
from models import VGG, LeNet

PATH = './models/model_cifar10.pth'    # 名前は適宜修正
model = VGG('VGG11')                   # 空のネットワークを準備（EX課題が終わっている場合）
# model = LeNet()                      # 空のネットワークを準備（EX課題が終わってない場合）
state_dict = torch.load(PATH)          # 保存したパラメタをロード
model.load_state_dict(state_dict)      # ネットワークにパラメタをセット
model.eval().cuda()                    # 評価モード & GPUへの移動

trainloader, testloader, classes = load_data(batch_size=4)
x, y = next(iter(testloader))          # ミニバッチを一つ取り出す
x, y, = x.cuda(), y.cuda()             # GPUに移す（modelの場合と違って，代入が必要）

---
# 1. 敵対的サンプルの生成

**Fast Gradient Sign Method (FGSM)** による敵対的サンプルの生成を見てみる．コメントを参考に，スライドで説明されたアイディアが実装されていることを確認せよ．

In [10]:
def fgsm(model, x, y, eps):
    '''
    model: モデル
    x    : 画像 or 画像のミニバッチ
    y    : ラベル or ラベルのミニバッチ
    eps  : 変化量の上限
    '''
    x0 = x.detach().clone()              # 元画像をとっておく

    x.requires_grad = True
    outputs = model(x)                   
    loss = F.cross_entropy(outputs, y)   # ロスを計算
    loss.backward()                      # 勾配を計算

    grad = x.grad.detach()               # 勾配を取得（detachはおまじないだと思って）
    grad = grad.sign()                   # 全ての値を正負によって+1or-1にする（FGSMのS要素）
    x = x0 + eps*grad                    # 計算した「悪い変化」を足しこむ

    x = x.clamp(0, 1)                    # 画素値を[0,1]の範囲に収める
    return x.detach()

### 練習問題１
上の関数を使って，敵対的サンプル`x_adv`を生成せよ．ただし，`eps=8/255`とせよ．攻撃された画像が分類されるクラス`y_adv`も計算せよ

### 解答

In [14]:
eps = 8/255                         # 攻撃の量
x_adv = fgsm(model, x, y, eps)      # 攻撃画像を手に入れる
y_adv = model(x_adv).argmax(dim=1)  # 攻撃後のラベルを手にいれる

### 可視化
以下の関数を使って，攻撃前後の画像を見てみる

In [49]:
# 複数の画像を並べて表示する関数
def list_images(title, images, tags, num_show=4, save_path=''):
    '''
    title       : 全体のタイトル
    images      : 画像のミニバッチ
    tags        : 画像につけるタグ（クラス名を想定）
    num_show    : 表示する画像の枚数
    save_path   : 保存先（figsディレクトリを作った上で，figs/images.pdfなど） 
    '''
    images = images.detach().cpu()               # !! プロットの際は，画像をCPUに移す
    fig, ax = plt.subplots(1, num_show, figsize=(10, 3))
    fig.suptitle(title, fontsize=12)
    for i in range(num_show):
        ax[i].set_title(tags[i])
        ax[i].imshow(images[i].permute(1,2,0))   # permuteで32x32x3にする
        ax[i].axis("off")                        # 余分な軸を消す
    plt.tight_layout()
    plt.show()
    
    if save_path: 
        plt.savefig(save_path)

In [ ]:
title = "Original images"
y_names = [classes[yi] for yi in y]  # クラスIDをクラス名に変換．この文法に関しては「Python リスト内包表記」で検索
list_images(title, x, y_names, num_show=4, save_path='')

title = "Attacked images (eps = 8/255)"
y_adv_names = [classes[yi] for yi in y_adv]
list_images(title, x_adv, y_adv_names, num_show=4, save_path='')

---
# 2. 敵対的サンプルの評価

攻撃の強度`eps`が大きくなっていくと，分類精度は落ちていくはずである．それを実際に観察したい．

精度を計算するのでそこそこのサンプル数が欲しい．なのでバッチサイズを大きくして改めてデータ読み込む

In [ ]:
trainloader, testloader, classes = load_data(batch_size=1000)  # バッチサイズ1000
x, y = next(iter(testloader))                                  # ミニバッチを一つ取り出す

### 練習問題２
FGSMの攻撃強度`eps`は攻撃強度を表す．様々な`eps`に対してネットワークの分類精度を計算せよ．

その結果をグラフとしてプロットせよ（横軸が`eps`，縦軸が分類精度）．

In [ ]:
eps_list = [0, 1/255, 2/255, 4/255, 8/255, 16/255]  # 横軸
acc_list = []                                       # 縦軸（これから計算）

for epsilon in eps_list:
    ##この中を埋める



    acc_list.append(acc_list.item())


plt.plot(eps_list, acc_list)    
plt.xlabel(' epsilon ')
plt.ylabel(' accuracy ')
plt.title(' classification accuracies under FGSM attack ')
plt.show()

#### 解答

In [ ]:
eps_list = [0, 1/255, 2/255, 4/255, 8/255, 16/255]  # 横軸
acc_list = []                                       # 縦軸（これから計算）


for epsilon in eps_list:
    adv_images = fgsm(model, x, y, epsilon)
    preds = model(adv_images).argmax(dim=-1)
    n_hits = (preds == y).sum()    # 正解の総数
    n_samples = len(y)             # サンプルの総数
    acc = n_hits / n_samples       # 精度
    acc_list.append(acc.item())    # .item()は tensor型をfloat型に変換している．どこかで一度だけ行えばいい．

plt.plot(eps_list, acc_list)    
plt.xlabel(' epsilon ')
plt.ylabel(' accuracy ')
plt.title(' classification accuracies under FGSM attack ')
plt.show()

### 練習問題３

関数`evaluate_fgsm(model, dataloader, epsilon)`を定義せよ．この関数は
```
acc = evaluate_fgsm(model, testloader, 0.03)
```
のように使って，データセット全体においてFSGMで攻撃された画像に対する分類精度を計算する．

これを設計した上で，練習問題2のように`eps_list`と正解率を軸とするプロットをせよ．訓練データ，テストデータそれぞれの分を重ねて一つのグラフにせよ（2本の折れ線が表示されるイメージ）．

プロットを重ねるには単純に以下のようにすればOK．
```
plt.plot(acc_list_train)
plt.plot(acc_list_test)
plt.show()
```

In [51]:
def evaluate_fgsm(model, dataloader, eps):
    
    n_hits = 0 
    n_samples = 0 
    for x, y in dataloader:
        x, y = x.cuda(), y.cuda() 
        '''
        練習問題２を参考にここを埋める
        '''
    
    acc = n_hits / n_samples
    return acc 

#### 解答

`test_fgsm()`は，Day 1で作った`test()`を微修正するだけで作れる．プロットに関しては以下のようになる．


In [ ]:
# 訓練データでの処理
acc_list_train = []
for epsilon in eps_list:
    acc = evaluate_fgsm(model, trainloader, epsilon)
    acc_list_train.append(acc)

# テストデータでの処理
acc_list_test = []
for epsilon in eps_list:
    acc = evaluate_fgsm(model, testloader, epsilon)
    acc_list_test.append(acc)

plt.plot(eps_list, acc_list_train, label='training')
plt.plot(eps_list, acc_list_test, label='test')
plt.legend()
plt.show()  

# 3. より強力な攻撃

FGSMは1-step attackである（勾配を一度だけ足しこむ）．ここではより強力な攻撃として，Iterative FGSM（I-FGSM）を考えてみる．I-FGSMは単純に，FGSMを複数回行う手法である．

### 練習課題４
I-FGSMを実装せよ．とりあえず「FGSMを繰り返し適用してより強い攻撃にする」という言葉から大体想像した通りのものを実装してみるので良い．

In [ ]:
def ifgsm(model, x, y, eps, n_steps):
    '''
    stesp: 何回FGSMを適用するか
    '''
    eps_per_step = eps / n_steps  # 1ステップあたりの攻撃上限．こちらを使ってFGSMを行う
    

### 解答

In [ ]:
def ifgsm(model, x, y, eps, n_steps):
    '''
    stesp: 何回FGSMを適用するか
    '''
    eps_per_step = eps / n_steps  # 1ステップあたりの攻撃上限．こちらを使ってFGSMを行う
    for i in range(n_steps):
        x = fgsm(model, x, y, eps_per_step)
    
    return x 

### 実験課題

I-FGSMのステップ数が1の場合（FGSMに相当）と，ステップ数>1の場合での攻撃性能の差を知りたい．
1. 練習問題3と同様に，`evaluate_ifgsm`を作成せよ．
2. それを用いて，練習問題２，３のような横軸が攻撃強度，縦軸が分類精度のグラフを描け．
ただし，
- プロットにあたっては`testloader`だけ考えれば良い
- `n_steps`を適当な範囲で汎化させて，ステップ数が増えるとどうなるのか観察せよ
- なぜそのようなことになるのか考察せよ